In [1]:
using Random
using LinearAlgebra
rng = MersenneTwister(2018)
import LinearAlgebra.dot
import LinearAlgebra.norm
import LinearAlgebra.triu
import LinearAlgebra.diag
using Printf

In [2]:
A = [2 -1 4; 3 4 -2; 1 4 2; -4 -1 3]
A = float(A)
A0 = copy(A)
B0 = copy(A)
C0 = copy(A)

4×3 Matrix{Float64}:
  2.0  -1.0   4.0
  3.0   4.0  -2.0
  1.0   4.0   2.0
 -4.0  -1.0   3.0

In [3]:
function house(x)
    """Computes the Householder transformation for input vector x"""
    sigma = dot(x[2:end],x[2:end])
    v = copy(x)

    if sigma == 0
        beta = 0
        return beta, v
    end

    sq = sqrt(x[1]^2 + sigma)
    if x[1] > 0
        v[1] += sq
    else
        v[1] -= sq
    end

    beta = 2.0 / (v[1]^2 + sigma)

    return beta, v
end


house (generic function with 1 method)

In [4]:
##Householder transformation
    m = size(A,1)
    n = size(A,2)
    vA = zeros(n)
    E = Matrix{Float64}(I, m, m)
    kend = (m > n ? n : m-1)
    #display(kend)
   Q = copy(E)
    for k=1:kend
        beta, v = house(A[k:end,k])
    G = Matrix{Float64}(I, m-k+1, m-k+1) - beta*v*v'

    P = copy(E)
    P = float(P)
    P[k:end, k:end] = G
 
    Q = Q*P
    
    #v = v/v(1)
        for j=k:n
            # vA = beta * v^T * A
            vA[j] = 0.0
            for i=k:m
                vA[j] += v[i-k+1] * A[i,j]
            end
            vA[j] *= beta
        end
        # A - beta v (v^T A)
        for j=k:n, i=k:m
            A[i,j] -= v[i-k+1] * vA[j]
        end
    v = v/v[1]
        A[k+1:end,k] = v[2:end]
        # Saving v in the lower triangular part of A.
        # This was not done here but one can always
        # divide v by v[1] such that v[1] = 1 is always true.
        # In that case, v[1] does not need to be stored.
    end
    R = triu(A)
    display("R")
    display(R)
    display("Q")
    display(Q)

"R"

4×3 Matrix{Float64}:
 -5.47723  -3.28634   1.46059
  0.0      -4.81664   0.45675
  0.0       0.0      -5.53697
  0.0       0.0       0.0

"Q"

4×4 Matrix{Float64}:
 -0.365148   0.45675   -0.781061   0.219065
 -0.547723  -0.45675    0.179047   0.677733
 -0.182574  -0.705887  -0.467599  -0.499742
  0.730297  -0.290659  -0.373145   0.492896

In [5]:
function givens(a, b)
    if b == 0
        c = 1
        s = 0
    else
        if abs(b) > abs(a)
            tau = -a/b
            s = 1.0/sqrt(1.0+tau*tau)
            c = s*tau
        else
            tau = -b/a
            c = 1.0/sqrt(1.0+tau*tau)
            s = c*tau
        end
    end
    return (c, s)
end

givens (generic function with 1 method)

In [6]:
q=Q[1,:]

4-element Vector{Float64}:
 -0.3651483716701107
  0.45675013919557
 -0.7810611209854971
  0.21906506407086604

In [7]:
n = size(q,1)
G = Matrix(1.0I, n, n)
# for k= (n-1):1
k=n-1
while (k>=1)
#     display(k)
    c, s = givens(q[k], q[k+1])
    T = Matrix(1.0I, n, n) # Identity matrix of Float64 type
    T[k,k] = T[k+1, k+1] = c; T[k,k+1]= -s
    T[k+1,k] = s; 
    G = T*G
#     display("T")
#     display(T)
    # Apply the Givens rotation to row k and k+1
    q[k], q[k+1] = ( c * q[k] - s * q[k+1], s * q[k] + c * q[k+1] )
#     display(q)
    k=k-1
#     display("G")
#     display(G)
end
    display("q")
    display(q)
    display("G")
    display(G)

"q"

4-element Vector{Float64}:
 -0.9999999999999996
  5.551115123125783e-17
  5.551115123125783e-17
 -2.7755575615628914e-17

"G"

4×4 Matrix{Float64}:
 0.365148  -0.45675    0.781061  -0.219065
 0.930949   0.179152  -0.306357   0.0859244
 0.0        0.871369   0.4724    -0.132494
 0.0        0.0        0.270051   0.962846

In [8]:
G #Givens rotarors

4×4 Matrix{Float64}:
 0.365148  -0.45675    0.781061  -0.219065
 0.930949   0.179152  -0.306357   0.0859244
 0.0        0.871369   0.4724    -0.132494
 0.0        0.0        0.270051   0.962846

In [9]:
H = G*R #Upper Hessenberg

4×3 Matrix{Float64}:
 -2.0       1.0      -4.0
 -5.09902  -3.92232   3.13786
  0.0      -4.19707  -2.21767
  0.0       0.0      -1.49526

In [10]:
R1 = H[[2,3,4],:] #Upper triangular R1 removing the first row of H

3×3 Matrix{Float64}:
 -5.09902  -3.92232   3.13786
  0.0      -4.19707  -2.21767
  0.0       0.0      -1.49526

In [11]:
Q2 = Q*G'

4×4 Matrix{Float64}:
 -1.0           9.26494e-17  -2.07915e-18  -2.4153e-17
  2.72721e-17  -0.588348     -0.403212      0.700904
  2.19326e-16  -0.196116     -0.769768     -0.60745
  2.66774e-16   0.784465     -0.494851      0.373815

In [12]:
Q1 = Q2[[2,3,4],[2,3,4]]

3×3 Matrix{Float64}:
 -0.588348  -0.403212   0.700904
 -0.196116  -0.769768  -0.60745
  0.784465  -0.494851   0.373815

In [13]:
A1 = Q1*R1 #after deleting first row of A

3×3 Matrix{Float64}:
  3.0   4.0  -2.0
  1.0   4.0   2.0
 -4.0  -1.0   3.0

In [14]:
B0 #given matrix A

4×3 Matrix{Float64}:
  2.0  -1.0   4.0
  3.0   4.0  -2.0
  1.0   4.0   2.0
 -4.0  -1.0   3.0